# Fire response traits in plants from field samples

This script contains examples of R code to query tables in the database

## Load libraries

In [1]:
library(RPostgreSQL)
library(ggplot2)
##library(forcats)
library(dplyr)
#library(data.table)
require(tidyr)

Loading required package: DBI


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: tidyr



## Connect to the database

Read database credentials

In [2]:
if (file.exists("../secrets/database.ini")) {
  tmp <- readLines("../secrets/database.ini")[-1]
  tmp <- strsplit(tmp,'=',fixed=2)
  dbinfo <- unlist(lapply(tmp,function(x) x[2]))
  names(dbinfo) <- unlist(lapply(tmp,function(x) x[1]))
  dbinfo <- data.frame(t(dbinfo),stringsAsFactors=F)
   rm(tmp)
} else {
  cat("No database information found")
}

Connection to the postgresql server (remember to update .pgpass file)

In [3]:
drv <- dbDriver("PostgreSQL") ## 
con <- dbConnect(drv, dbname = dbinfo$database,
                 host = dbinfo$host, port = dbinfo$port,
                 user = dbinfo$user)

In [4]:
qry <- 
'SELECT family, genus, "speciesID" as spp, 
count(distinct q.record_id) as nquadrat, 
count(distinct g8.record_id) as germ8, 
count(distinct r2.record_id) as rect2,
count(distinct g1.record_id) as germ1, 
count(distinct gw1.record_id) as grow1, 
count(distinct r4.record_id) as repr4, 
count(distinct s5.record_id) as surv5, 
count(distinct s6.record_id) as surv6, 
count(distinct s7.record_id) as surv7, 
count(distinct d1.record_id) as disp1, 
count(distinct r3a.record_id) as repr3a, 
count(distinct r3.record_id) as repr3, 
count(distinct s4.record_id) as surv4, 
count(distinct s1.record_id) as surv1
FROM species.caps  
LEFT JOIN form.quadrat_samples q  
 ON "speciesCode_Synonym"=q.species_code::text   
LEFT JOIN litrev.germ8 g8  ON "speciesCode_Synonym"=g8.species_code::text   
LEFT JOIN litrev.rect2 r2  ON "speciesCode_Synonym"=r2.species_code::text   
LEFT JOIN litrev.germ1 g1  ON "speciesCode_Synonym"=g1.species_code::text   
LEFT JOIN litrev.grow1 gw1  ON "speciesCode_Synonym"=gw1.species_code::text   
LEFT JOIN litrev.repr4 r4  ON "speciesCode_Synonym"=r4.species_code::text   
LEFT JOIN litrev.surv5 s5  ON "speciesCode_Synonym"=s5.species_code::text   
LEFT JOIN litrev.surv6 s6  ON "speciesCode_Synonym"=s6.species_code::text   
LEFT JOIN litrev.surv7 s7  ON "speciesCode_Synonym"=s7.species_code::text   
LEFT JOIN litrev.disp1 d1  ON "speciesCode_Synonym"=d1.species_code::text   
LEFT JOIN litrev.repr3a r3a  ON "speciesCode_Synonym"=r3a.species_code::text   
LEFT JOIN litrev.repr3 r3  ON "speciesCode_Synonym"=r3.species_code::text   
LEFT JOIN litrev.surv4 s4  ON "speciesCode_Synonym"=s4.species_code::text   
LEFT JOIN litrev.surv1 s1  ON "speciesCode_Synonym"=s1.species_code::text   

GROUP BY spp'

In [5]:
spp.table<- dbGetQuery(con, qry)
str(spp.table)

'data.frame':	15732 obs. of  17 variables:
 $ family  : chr  "Brassicaceae" "Myrtaceae" "Myrtaceae" "Apiaceae" ...
 $ genus   : chr  "Lepidium" "Eucalyptus" "Melaleuca" "Actinotus" ...
 $ spp     : num  2358 2359 2360 2361 2362 ...
 $ nquadrat: num  0 0 0 0 0 2 1 0 0 0 ...
 $ germ8   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ rect2   : num  0 1 0 2 0 0 0 0 0 0 ...
 $ germ1   : num  0 1 0 1 0 0 0 0 0 1 ...
 $ grow1   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ repr4   : num  0 0 0 1 0 0 0 0 0 0 ...
 $ surv5   : num  0 0 0 2 0 0 0 0 0 0 ...
 $ surv6   : num  0 0 0 0 0 0 0 0 0 0 ...
 $ surv7   : num  0 0 0 1 0 0 0 0 0 0 ...
 $ disp1   : num  3 0 0 3 1 1 0 1 0 1 ...
 $ repr3a  : num  0 0 0 0 0 0 1 0 0 0 ...
 $ repr3   : num  0 0 0 3 0 0 0 0 0 0 ...
 $ surv4   : num  0 0 0 0 0 0 1 0 0 0 ...
 $ surv1   : num  1 6 3 14 1 1 3 1 0 7 ...


In [6]:
saveRDS(file='../data/Summary-traits-species.rds',spp.table)

In [7]:
dbDisconnect(con)

[1] TRUE